# Test lógico negación

In [1]:
import sys

sys.path.append("../src")

In [2]:
import nltk
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm
from pyprover import *

from nltk.sem.logic import LogicParser

lp = LogicParser()

In [3]:
from config.config import PATHS
from parser.parser_utils import Parse2pyprover

In [5]:
data_folder = PATHS['training_data_folder']
data_path = Path(data_folder, 'negacion_5_fol.csv')
data = pd.read_csv(data_path, names=['Sentence 1', 'Sentence 2', 'Relation'], sep=";", nrows=2)
print(f'Data shape: {data.shape}')
data.head()

Data shape: (2, 3)


,Sentence 1,Sentence 2,Relation
0,(exists x.MUEBLE(x) & all x.(MUEBLE(x) -> FLOT...,exists x.(MUEBLE(x) & -FLOTA(x)),1
1,(exists x.MUEBLE(x) & all x.(MUEBLE(x) -> CALL...,exists x.(MUEBLE(x) & -CALLAR(x)),1


In [6]:
sentences1 = data['Sentence 1'].tolist()
sentences2 = data['Sentence 2'].tolist()

In [30]:
def test_negacion(sentence1:str, sentence2:str) -> bool:
    '''
    Test negation between two sentences.
    '''
    #Test sentence1 implies -sentence2
    sentence1_nltk = lp.parse(f'{sentence1}')
    # print(f'Sentence 1 NLTK: {sentence1_nltk}')
    sentence1_pyprover = Parse2pyprover.parse(sentence1_nltk)

    negated_sentence2_nltk = lp.parse(f'-{sentence2}')
    # print(f'Negación Sentence 2 NLTK: {negated_sentence2_nltk}')
    negated_sentence2_pyprover = Parse2pyprover.parse(negated_sentence2_nltk)

    premisas = [sentence1_pyprover]
    conclusion = negated_sentence2_pyprover
    resultado1 = proves(premisas, conclusion)
    # print('Resultado:', resultado1)

    # Test -sentence1 implies sentence2
    sentence2_nltk = lp.parse(f'{sentence2}')
    # print(f'Sentence 2 NLTK: {sentence2_nltk}')
    sentence2_pyprover = Parse2pyprover.parse(sentence2_nltk)

    negated_sentence1_nltk = lp.parse(f'-{sentence1}')
    # print(f'Negación Sentence 1 NLTK: {negated_sentence1_nltk}')
    negated_sentence1_pyprover = Parse2pyprover.parse(negated_sentence1_nltk)


    premisas = [sentence2_pyprover]
    conclusion = negated_sentence1_pyprover
    resultado2 = proves(premisas, conclusion)
    # print('Resultado:', resultado2)

    return resultado1 and resultado2

In [31]:
tested = list()
for index, row in data.iterrows():
    sentences1 = row['Sentence 1']
    sentences2 = row['Sentence 2']
    print('Sentences:', sentences1, sentences2)
    relation = row['Relation']
    result = test_negacion(sentences1, sentences2)
    tested.append(int(result))

data['Tested'] = tested
data.head()

Sentences: (exists x.MUEBLE(x) & all x.(MUEBLE(x) -> FLOTA(x))) exists x.(MUEBLE(x) & -FLOTA(x))
Sentences: (exists x.MUEBLE(x) & all x.(MUEBLE(x) -> CALLAR(x))) exists x.(MUEBLE(x) & -CALLAR(x))


,Sentence 1,Sentence 2,Relation,Tested
0,(exists x.MUEBLE(x) & all x.(MUEBLE(x) -> FLOT...,exists x.(MUEBLE(x) & -FLOTA(x)),1,1
1,(exists x.MUEBLE(x) & all x.(MUEBLE(x) -> CALL...,exists x.(MUEBLE(x) & -CALLAR(x)),1,1


In [32]:
data[data['Tested'] != data['Relation']]

,Sentence 1,Sentence 2,Relation,Tested
